In [3]:
import pandas as pd
import sqlite3
from datetime import datetime

# -----------------
# 1. MOCK FX RATES
# -----------------

fx_rates = pd.DataFrame({
    "Currency": ["EUR", "INR", "GBP", "JPY"],
    "Rate": [1.08, 0.012, 1.25, 0.0067],  # USD conversion rates (example)
    "Date": [datetime.today().strftime('%Y-%m-%d')]*4
})

print("Mock FX Rates:")
display(fx_rates)

# -----------------
# 2. CREATE DATABASE
# -----------------

conn = sqlite3.connect("fx_risk.db")
cursor = conn.cursor()

# Create FX Rates Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS fx_rates (
    date TEXT,
    currency TEXT,
    rate REAL
)
""")

# Insert Data
cursor.execute("DELETE FROM fx_rates")  # clear old data
for _, row in fx_rates.iterrows():
    cursor.execute("INSERT INTO fx_rates VALUES (?, ?, ?)",
                   (row["Date"], row["Currency"], row["Rate"]))
conn.commit()

# -----------------
# 3. MOCK EXPOSURES
# -----------------

exposures = pd.DataFrame({
    "Currency": ["EUR", "INR", "GBP", "JPY"],
    "Exposure": [100000, 5000000, 75000, 12000000]
})

cursor.execute("""
CREATE TABLE IF NOT EXISTS exposures (
    currency TEXT,
    exposure REAL
)
""")
cursor.execute("DELETE FROM exposures")
for _, row in exposures.iterrows():
    cursor.execute("INSERT INTO exposures VALUES (?, ?)",
                   (row["Currency"], row["Exposure"]))
conn.commit()

# -----------------
# 4. JOIN & CALCULATE
# -----------------

query = """
SELECT e.currency, e.exposure, f.rate,
       e.exposure * f.rate AS exposure_usd
FROM exposures e
JOIN fx_rates f
ON e.currency = f.currency
WHERE f.date = (SELECT MAX(date) FROM fx_rates)
"""

df = pd.read_sql_query(query, conn)
print("FX Exposure in USD:")
display(df)

# Scenario Analysis
df["P&L_If_1pct_Move"] = df["exposure_usd"] * 0.01
display(df)


Mock FX Rates:


,Currency,Rate,Date
0,EUR,1.0800,2025-09-18
1,INR,0.0120,2025-09-18
2,GBP,1.2500,2025-09-18
3,JPY,0.0067,2025-09-18


FX Exposure in USD:


,currency,exposure,rate,exposure_usd
0,EUR,100000.0,1.0800,108000.0
1,INR,5000000.0,0.0120,60000.0
2,GBP,75000.0,1.2500,93750.0
3,JPY,12000000.0,0.0067,80400.0


,currency,exposure,rate,exposure_usd,P&L_If_1pct_Move
0,EUR,100000.0,1.0800,108000.0,1080.0
1,INR,5000000.0,0.0120,60000.0,600.0
2,GBP,75000.0,1.2500,93750.0,937.5
3,JPY,12000000.0,0.0067,80400.0,804.0
